# Extending the TrainedModel Table to support Transfer of multiple Stages

In [1]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
dj.config['schema_name'] = "nnfabrik_example"

from nnfabrik.main import *
from nnfabrik.templates.transfer import TransferredTrainedModelBase
from nnfabrik.templates.transfer.recipes import *

Connecting anix@134.2.168.16:3306


In [38]:
schema.drop()

Proceed to delete entire schema `nnfabrik_example`? [yes, No]:  yes


In [2]:
@schema
class TransferredTrainedModel(TransferredTrainedModelBase):
    table_comment = "Transferred trained models"

## (1) Normal Training

### Add some entries to the table (usual step)

In [3]:
fabrikant_info = dict(fabrikant_name="Your Name", email="your@email.com", affiliation="thelab", dj_username="yourname")
Fabrikant().insert1(fabrikant_info)

In [4]:
Seed().insert([{'seed':7}])

In [5]:
dataset_fn = "nnfabrik.examples.mnist.dataset.mnist_dataset_fn"

dataset_config = dict(batch_size=64) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="A comment about the dataset!");

In [6]:
# specify model function as string (the function must be importable) as well as the model config
model_fn = "nnfabrik.examples.mnist.model.mnist_model_fn"
model_config = dict(h_dim=5) # we specify all the inputs except the ones required by nnfabrik

Model().add_entry(model_fn=model_fn, model_config=model_config, 
                  model_fabrikant="Your Name", model_comment="A comment about the model!");

In [7]:
# specify trainer function as string (the function must be importable) as well as the trainer config
trainer_fn = "nnfabrik.examples.mnist.trainer.mnist_trainer_fn"
trainer_config = dict(epochs=1) # we specify all the inputs except the ones required by nnfabrik

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, 
                  trainer_fabrikant="Your Name", trainer_comment="A comment about the trainer!");

### Populating TrainedModel normally

In [8]:
TransferredTrainedModel.populate()

100%|██████████| 938/938 [00:20<00:00, 45.37it/s]


In [9]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,collapsed_history,data_transfer,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,,,,,,,,0,A comment about the trainer!.A comment about the model!.A comment about the dataset!,73.7033,=BLOB=,None,2020-11-04 22:42:55,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0


Now that we have trained a model on MNIST, we want to transfer it.
For this, we consider two different scenarios, both of which can be fully automated in our transfer framework.

In both cases, the operations boil down to:
* what datasets and/or models and/or trainers you would like to use at a specific transfer step -> specify this in the recipe
* which component should be handed over between two consecutive transfer steps (data and/or model state) -> specify this in the recipe and use an appropriate "trainer"
* which already-exisiting entries you would like this transfer step to be applied on -> specify this in the `populate` restrictions

## (2) Model-state Transfer

### Add the recipes for transfer

First, let's add the dataset that we want to use for the transfer. 

In [10]:
dataset_fn = "nnfabrik.examples.mnist.dataset.mnist_dataset_fn"

dataset_config = dict(batch_size=64, apply_augmentation=True) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="Augmented MNIST");

In [11]:
Dataset()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion
nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,=BLOB=,Your Name,Augmented MNIST,2020-11-04 22:42:56
nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,=BLOB=,Your Name,A comment about the dataset!,2020-11-04 22:42:34


Now, we define the recipe.

In [12]:
transfer_from = {"dataset_fn": 'nnfabrik.examples.mnist.dataset.mnist_dataset_fn', "dataset_hash": '9aee736870714f8b7c3cc084087ce886'}
transfer_to = {"dataset_fn": 'nnfabrik.examples.mnist.dataset.mnist_dataset_fn', "dataset_hash": '28aefc2308569727c6017c66c9122d77'}

In [13]:
DatasetTransferRecipe().add_entry(transfer_from=transfer_from, transfer_to=transfer_to, transfer_step=1)

In [14]:
DatasetTransferRecipe()

transfer_step,data_transfer,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,prev_dataset_fn name of the dataset loader function,prev_dataset_hash hash of the configuration object
1,0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886


### Assign the recipes to the table

In [15]:
TransferredTrainedModel.transfer_recipe = [DatasetTransferRecipe()]

### Populate the table for the first transfer step 

In [16]:
TransferredTrainedModel.populate()

100%|██████████| 938/938 [00:21<00:00, 44.66it/s]


In [17]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,collapsed_history,data_transfer,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,,,,,,,,0,A comment about the trainer!.A comment about the model!.A comment about the dataset!,73.7033,=BLOB=,None,2020-11-04 22:42:55,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0
1,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,abebf792503627fe90e131a0894309c1,0,A comment about the trainer!.A comment about the model!.Augmented MNIST,75.9933,=BLOB=,None,2020-11-04 22:43:18,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0


## (3) Data Transfer (Knowledge Distillation)

Now that we have seen a simple transfer of the model state between two training runs on similar datasets, let's consider a more challenging scenario.

Assume we want to transfer knowledge between two slightly different models (e.g. with different hidden size) and at the same time have a domain shift in the data. 

### Add the recipes for the intermediate transfer step (dataset generation)

For this step, we are keep everything the same, but replace the trainer by a "pseudo-trainer" that simply captures and saves the logits for the whole dataset.

In [18]:
trainer_fn = "nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn"

trainer_config = dict(batch_size=64, apply_augmentation=False) 

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, 
                    trainer_fabrikant="Your Name", trainer_comment="Transfer MNIST Logits");

In [19]:
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,=BLOB=,Your Name,A comment about the trainer!,2020-11-04 22:42:34
nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn,ab91f734757071bf0b98ab74c6e8583c,=BLOB=,Your Name,Transfer MNIST Logits,2020-11-04 22:43:18


In [20]:
transfer_from = {"trainer_fn": 'nnfabrik.examples.mnist.trainer.mnist_trainer_fn', "trainer_hash": '79e921430b7f44a205d79d0087b59dc0'}
transfer_to = {"trainer_fn": 'nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn', "trainer_hash": 'ab91f734757071bf0b98ab74c6e8583c'}

In [21]:
TrainerTransferRecipe().add_entry(transfer_from=transfer_from, transfer_to=transfer_to, transfer_step=1, data_transfer=True)

In [22]:
TransferredTrainedModel.transfer_recipe = [TrainerTransferRecipe()]

### Populate the table for the first transfer step 

In [23]:
TransferredTrainedModel.populate()

100%|██████████| 157/157 [00:02<00:00, 59.97it/s]


In [24]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,collapsed_history,data_transfer,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,,,,,,,,0,A comment about the trainer!.A comment about the model!.A comment about the dataset!,73.7033,=BLOB=,None,2020-11-04 22:42:55,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0
1,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,abebf792503627fe90e131a0894309c1,0,A comment about the trainer!.A comment about the model!.Augmented MNIST,75.9933,=BLOB=,None,2020-11-04 22:43:18,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0
1,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn,ab91f734757071bf0b98ab74c6e8583c,7,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,abebf792503627fe90e131a0894309c1,1,Transfer MNIST Logits.A comment about the model!.A comment about the dataset!,-1.0,=BLOB=,None,2020-11-04 22:43:37,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn,ab91f734757071bf0b98ab74c6e8583c


### Add the recipes for the second transfer step (using the generated dataset)

Now we need a dataset that loads the transferred dataset and a trainer that can handle it. Let's introduce those here.

In [25]:
trainer_fn = "nnfabrik.examples.mnist_transfer.trainer.mnist_trainer_fn"

trainer_config = dict(batch_size=64, apply_augmentation=False) 

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, 
                    trainer_fabrikant="Your Name", trainer_comment="Use Transferred MNIST Logits");

In [26]:
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,=BLOB=,Your Name,A comment about the trainer!,2020-11-04 22:42:34
nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn,ab91f734757071bf0b98ab74c6e8583c,=BLOB=,Your Name,Transfer MNIST Logits,2020-11-04 22:43:18
nnfabrik.examples.mnist_transfer.trainer.mnist_trainer_fn,ab91f734757071bf0b98ab74c6e8583c,=BLOB=,Your Name,Use Transferred MNIST Logits,2020-11-04 22:43:38


In [27]:
dataset_fn = "nnfabrik.examples.mnist_transfer.dataset.mnist_dataset_fn"

dataset_config = dict(batch_size=64, apply_augmentation=True) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="Augmented MNIST with Knowledge Distallation");

In [28]:
Dataset()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion
nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,=BLOB=,Your Name,Augmented MNIST,2020-11-04 22:42:56
nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,=BLOB=,Your Name,A comment about the dataset!,2020-11-04 22:42:34
nnfabrik.examples.mnist_transfer.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,=BLOB=,Your Name,Augmented MNIST with Knowledge Distallation,2020-11-04 22:43:38


In [29]:
transfer_from = {"trainer_fn": 'nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn', 
                 "trainer_hash": 'ab91f734757071bf0b98ab74c6e8583c',
                 "dataset_fn": 'nnfabrik.examples.mnist.dataset.mnist_dataset_fn',
                 "dataset_hash": '9aee736870714f8b7c3cc084087ce886'
                }
transfer_to = {"trainer_fn": 'nnfabrik.examples.mnist_transfer.trainer.mnist_trainer_fn', 
                 "trainer_hash": 'ab91f734757071bf0b98ab74c6e8583c',
                 "dataset_fn": 'nnfabrik.examples.mnist_transfer.dataset.mnist_dataset_fn',
                 "dataset_hash": '28aefc2308569727c6017c66c9122d77'
                }

In [30]:
TrainerDatasetTransferRecipe().add_entry(transfer_from=transfer_from, transfer_to=transfer_to, transfer_step=2, data_transfer=False)

In [31]:
TransferredTrainedModel.transfer_recipe = [TrainerDatasetTransferRecipe()]

### Populate the table for the second transfer step 

In [32]:
TransferredTrainedModel.populate()

100%|██████████| 938/938 [00:17<00:00, 53.30it/s]


In [33]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,collapsed_history,data_transfer,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,,,,,,,,0,A comment about the trainer!.A comment about the model!.A comment about the dataset!,73.7033,=BLOB=,None,2020-11-04 22:42:55,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0
1,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,7,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,abebf792503627fe90e131a0894309c1,0,A comment about the trainer!.A comment about the model!.Augmented MNIST,75.9933,=BLOB=,None,2020-11-04 22:43:18,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0
1,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn,ab91f734757071bf0b98ab74c6e8583c,7,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist.trainer.mnist_trainer_fn,79e921430b7f44a205d79d0087b59dc0,abebf792503627fe90e131a0894309c1,1,Transfer MNIST Logits.A comment about the model!.A comment about the dataset!,-1.0,=BLOB=,None,2020-11-04 22:43:37,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn,ab91f734757071bf0b98ab74c6e8583c
2,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist_transfer.dataset.mnist_dataset_fn,28aefc2308569727c6017c66c9122d77,nnfabrik.examples.mnist_transfer.trainer.mnist_trainer_fn,ab91f734757071bf0b98ab74c6e8583c,7,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist.dataset.mnist_dataset_fn,9aee736870714f8b7c3cc084087ce886,nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn,ab91f734757071bf0b98ab74c6e8583c,9b07c14f933a55a38ee849bb41027e71,0,Use Transferred MNIST Logits.A comment about the model!.Augmented MNIST with Knowledge Distallation,0.393618,=BLOB=,None,2020-11-04 22:44:15,nnfabrik.examples.mnist.model.mnist_model_fn,24922759b843076328c4f3b9df3f88d0,nnfabrik.examples.mnist_transfer.dataset.mnist